In [279]:
import time
import datetime
import random
from sys import maxsize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from sklearn.linear_model import Lasso
from sklearn import preprocessing, svm
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from IPython.display import display, HTML

# Utilities

In [386]:
"""
Sign of learningRate determines whether to use gradient ascent/descent:
Negative learningRates = descent, positive = ascent
"""
def SGO(x, y, seed, gradientFunc, learningRate, error, maxIter):
    x = np.array(x)
    y = np.array(y)

    # Reshape 1-D arrays to column format
    # if len(x.shape) == 1:
    #     x = x.reshape(-1, 1)

    N = len(x)
    currentError = maxsize
    lastError = 0
    beta = np.array(seed)
#     print('SGO weight shape:', beta.shape)
    i = 0

    while i < maxIter:
        gradient = gradientFunc(beta, x, y)
#         yPredicted = x.dot(beta)
#         sqErrGradient = np.array(np.dot(x.T, (yPredicted - y)) / N)
        beta += learningRate * gradient
#         currentError = np.sum(np.square(y - yPredicted)) / N
        currentError = np.sum(gradient)
#         print(currentError)

        if abs(lastError - currentError) < error:
            break
        lastError = currentError
        i += 1

    return beta

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dSigmoid(y):
    return y * (1.0 - y)

def tanh(x):
    return np.tanh(x)

def dTanh(y):
    return 1 - y * y

# Neural Network

In [406]:
class NeuralNetwork:
    """
    inputNeurons: number of input neurons
    hiddenLayerShape: (number of layers, number of neurons per layer) tuple
    outputNeurons: number of output neurons
    """
    def __init__(self, inputNeurons, hiddenLayerShape, outputNeurons,
                 learnRate=1e-3, T=1, sigmoid=sigmoid, dSigmoid=dSigmoid):
        self.inputNeurons = inputNeurons + 1
        self.hiddenLayerShape = hiddenLayerShape
        self.outputNeurons = outputNeurons
        self.T = T
        self.sigmoid = sigmoid
        self.dSigmoid = dSigmoid
        hiddenLayers, hiddenNeurons = hiddenLayerShape
        
        input_range = 1.0 / self.inputNeurons ** (1/2)
        output_range = 1.0 / hiddenNeurons ** (1/2)
        self.w_input = np.random.normal(loc=0, scale=input_range,
                                   size=(self.inputNeurons, hiddenNeurons))
        self.w_hidden = np.random.normal(loc=0, scale=output_range,
                                        size=(hiddenLayers, hiddenNeurons, hiddenNeurons))
        self.w_output = np.random.normal(loc=0, scale=output_range,
                                   size=(hiddenNeurons, self.outputNeurons))
        
        self.a_input = np.ones(self.inputNeurons, dtype=float)
        self.a_hidden = np.ones(shape=(hiddenLayers, hiddenNeurons), dtype=float)
        self.a_output = np.ones(self.outputNeurons, dtype=float)
        
#         print('Input weights:', self.w_input)
#         print('Hidden weights:', self.w_hidden)
#         print('Output weights:', self.w_output)
        
        self.c_input = np.zeros(shape=(self.inputNeurons, hiddenNeurons), dtype=float)
        self.c_hidden = np.zeros(shape=(hiddenLayers, hiddenNeurons, hiddenNeurons), dtype=float)
        self.c_output = np.zeros(shape=(hiddenNeurons, self.outputNeurons), dtype=float)
        
        self.learnRate = learnRate
        self._currentLearnRate = learnRate
    
    def fit(self, x, y, iterations=100):
        stepSize = step = len(x) // 10
        xy = list(zip(x, y))
        self._currentLearnRate = self.learnRate
        
        for i in range(iterations):
            random.shuffle(xy)
            error = 0.0
            for xVal, yVal in xy:
                self._feedForward(xVal)
                error = self._backPropagate(yVal)
            if i == step:
                print('Error={}, current learning rate={}'.format(
                        error, self._currentLearnRate))
                step += stepSize
            
            self._currentLearnRate = self.learnRate / (1 + (i / self.T))
    
    def predict(self, x):
        predictions = []
        for xVal in x:
            predict = self._feedForward(xVal)
            predict[np.argmax(predict)] = 1
            predict[predict < 1.0] = 0
            print(predict)
            predictions.append(predict)
        return np.array(predictions, dtype=int)
    
    def score(self, x, y):
        if type(x) != np.ndarray:
            x = np.array(x)
        if type(y) != np.ndarray:
            y = np.array(y)
            
        correct = 0
        predictions = self.predict(x)
        precision, recall, f1, support = precision_recall_fscore_support(
            y, predictions, average='macro')
        
        for prediction, yTruth in zip(predictions, y):
            if np.array_equal(prediction, yTruth):
                correct += 1
                
        accuracy = correct / len(y)
#         print('Predictions:', predictions)
#         print('Truth:', y)
        
        return accuracy, precision, recall, f1
    
    def _feedForward(self, x):
        if type(x) != np.ndarray:
            x = np.array(x)
        
        self.a_input[:-1] = x
#         for i in range(len(self.a_input) - 1):
#             self.a_input[i] = self.w_input[i]
#         print('Input activation:', self.a_input)
        hiddenLayers, hiddenNeurons = self.hiddenLayerShape
#         print(self.w_hidden[0])
        
        if hiddenLayers > 0:
#             self.a_hidden[0] = sigmoid(self.a_input * self.w_input)
            for j in range(hiddenNeurons):
                activation = 0.0
                
                for i in range(self.inputNeurons):
                    activation += self.a_input[i] * self.w_input[i][j]
                self.a_hidden[0][j] = tanh(activation)
        
            for layer in range(1, hiddenLayers):
                for j in range(hiddenNeurons):
                    activation = 0.0

                    for i in range(hiddenNeurons):
#                         print('A_Hidden[{}][{}] len={}'.format(layer, i, len(self.a_hidden[layer][i])))
                        activation += self.a_hidden[layer][i] * self.w_hidden[layer][i][j]
                    self.a_hidden[layer][j] = tanh(activation)
#         print('Hidden activations:', self.a_hidden)
#         print('Output weights:', self.w_output)
#         print('Final hidden activations:', self.a_hidden[-1][0])
#         print('Final hidden activation weighted sums:', np.sum())
#         print('w_output[0]:', self.w_output[0])
        
        for k in range(self.outputNeurons):
            activation = 0.0
            
#             self.a_output = sigmoid(np.sum(self.a_hidden[-1] * self.w_output[:][k], axis=0))
            for j in range(hiddenNeurons):
                activation += self.a_hidden[-1][j] * self.w_output[j][k]
            
            self.a_output[k] = sigmoid(activation)
#         print('Output activations:', self.a_output)
        return self.a_output[:]
    
    """
    Update weights and return the current error
    @param y must be one-hot encoded representation of class
    """
    def _backPropagate(self, y):
        if type(y) != np.ndarray:
            y = np.array(y)
        
        outputDeltas = self.dSigmoid(self.a_output) * (self.a_output - y)
#         print('Output deltas:', outputDeltas)
        
        hiddenLayers, hiddenNeurons = self.hiddenLayerShape
        hiddenDeltas = np.zeros(shape=self.a_hidden.shape, dtype=float)
#         print('Hidden deltas:', hiddenDeltas)
#         print('Last hidden deltas:', hiddenDeltas[-1])
        
        for j in range(hiddenNeurons):
            error = np.sum(outputDeltas * self.w_output[j])
            hiddenDeltas[-1][j] = dTanh(self.a_hidden[-1][j]) * error
#         print('Hidden deltas from output error:', hiddenDeltas)
        
        for j in range(hiddenNeurons):
            change = outputDeltas * self.a_hidden[-1][j]
            self.w_output[j] -= self._currentLearnRate * change + self.c_output[j]
            self.c_output[j] = change
        
#         print('C_Output:', self.c_output)
        # TODO: Finish hidden layer error propagation for multiple hidden layers
        for layer in reversed(range(hiddenLayers - 1)):
            print('Backpropping through layer', layer)
            error = np.sum(hiddenDeltas[layer + 1] * self.w_hidden[layer], axis=1)
            hiddenDeltas[layer] = error
#         print('Hidden deltas:', hiddenDeltas)
        
        for i in range(self.inputNeurons):
            change = hiddenDeltas[0] * self.a_input[i]
            self.w_input[i] -= self._currentLearnRate * change * self.c_input[i]
            self.c_input[i] = change
            
#         print('C_Input:', self.c_input)
        error = np.sum(0.5 * np.square(self.a_output - y))
#         print('Overall error:', error)
#         print('new learning rate:', self._currentLearnRate)
        return error

# Predicting Malignancy of Breast Cancer Cases
## Source: [UCI ML Repository](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29)

In [407]:
breastCancerDf = pd.read_csv('data/breast-cancer-wisconsin.csv')
breastCancerDf['Class_2'] = breastCancerDf.Class.apply(lambda x: 1 if x == 2 else 0)
breastCancerDf['Class_4'] = breastCancerDf.Class.apply(lambda x: 1 if x == 4 else 0)
breastCancerDf.drop(breastCancerDf[breastCancerDf['BareNuclei'] == '?'].index, inplace=True)
yColumn = ['Class_2', 'Class_4']
xColumns = [col for col in breastCancerDf.columns
            if col != 'ID' and col not in yColumn
           and col != 'BareNuclei' and col != 'Class']
display(breastCancerDf[xColumns + yColumn].head())

print('% malignant', len(breastCancerDf['Class'].loc[breastCancerDf['Class_4'] == 1]) / len(breastCancerDf['Class_4']))

# Split into test and training sets
xTrain, xTest, yTrain, yTest = train_test_split(breastCancerDf[xColumns].as_matrix(),
                                               breastCancerDf[yColumn].as_matrix(),
                                               test_size=1/3, random_state=int(time.time()))

,ClumpThickness,CellSizeUniformity,CellShapeUniformity,MarginalAdhesion,SingleEpithelialCellSize,BlandChromatin,NormalNucleoli,Mitoses,Class_2,Class_4
0,5,1,1,1,2,3,1,1,1,0
1,5,4,4,5,7,3,2,1,1,0
2,3,1,1,1,2,3,1,1,1,0
3,6,8,8,1,3,3,7,1,1,0
4,4,1,1,3,2,3,1,1,1,0


% malignant 0.34992679355783307


In [412]:
nn = NeuralNetwork(8, (1, 8), 2, learnRate=1/np.sqrt(xTrain.shape[0]))
nn.fit(xTrain, yTrain)
accuracy, precision, recall, f1 = nn.score(xTest, yTest)
print('Accuracy: {:.4f}, precision: {:.4f}, recall: {:.4f}, F1: {:.4f}'.format(
        accuracy, precision, recall, f1))

Error=0.3910532404278619, current learning rate=0.0010417938465299898
Error=0.300196586471274, current learning rate=0.0005208969232649949
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[ 1.  0.]
[

/opt/conda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
